In [6]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [7]:
api_key = "e30eb1259555481b91906de4cb5a3cd5"
azure_endpoint = "https://artirst-assistant.openai.azure.com/"
api_version = "2024-02-15-preview"
llm = AzureOpenAI(
    model="gpt-3.5-turbo-instruct",
    deployment_name="35instruct",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="emb3large",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [8]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [9]:
from llama_index.core.evaluation import DatasetGenerator, QueryResponseDataset
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.readers.file import FlatReader, HTMLTagReader
from llama_index.core.node_parser import HTMLNodeParser, SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import set_global_handler
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SemanticSplitterNodeParser

from pathlib import Path

reader = SimpleDirectoryReader("test_data/")

docs = reader.load_data()

In [6]:
print(len(docs))

1


In [7]:
docs

[Document(id_='ece0de1c-27df-4aa7-944f-554e783e794a', embedding=None, metadata={'file_path': '/home/dngback/Desktop/artist_assistant/test_data/13400_page_1.html', 'file_name': '/home/dngback/Desktop/artist_assistant/test_data/13400_page_1.html', 'file_type': 'text/html', 'file_size': 2626, 'creation_date': '2024-03-06', 'last_modified_date': '2024-03-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='<div>\n  <h1 class="c-ttl-article__txt">結婚12年目「倖田來未」が明かす“夫と息子”との私生活\u3000「親子でプロレスにハマってます」</h1>\n  <p>\n    今年でデビュー23年目を迎える歌手の倖田來未（40）。「前編」では、下積み時代から芸能界における浮き沈みを経験しながら、「エロかっこいい」と称される人気歌手に上り詰めるまでの半生を聞いた。そんな彼女は1月18日にMusic ＆\n    Live Package「WINGS」をリリースした。収録曲の作曲には夫も参加しているが、今回の「後編」では主に夫や息子など家族がアーティスト・倖田來未にどのような影響を与えているかをインタビューした。\n  </p>\n  <p>\n    倖田來未が1月18

In [8]:
# Indexing documentation 
index = VectorStoreIndex.from_documents(docs)

INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"


In [12]:
query_engine = index.as_query_engine()
response = query_engine.query("倖田來未さんは結婚して何年目ですか？")
print(response)

INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
12年目


In [17]:
query_engine = index.as_query_engine()
response = query_engine.query("Who is Koda Kumi and how long has she been in the music industry?")
print(response)

INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"

Koda Kumi is a 40-year-old singer who has been in the music industry for 23 years.


In [16]:
query_engine = index.as_query_engine()
response = query_engine.query("倖田來未とは何者で、音楽業界でどれくらい働いていますか?")
print(response)

INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"

倖田來未は歌手であり、今年でデビュー23年目を迎えています。


---

### Test Part 2: Deeper

In [10]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [11]:
api_key = "a83e2f795b9f4aada15fba7f6594d216"
azure_endpoint = "https://artist-assistant-uk-south.openai.azure.com/"
api_version = "2024-02-15-preview"
llm = AzureOpenAI(
    model="gpt-3.5-turbo-instruct",
    deployment_name="35turbo16k",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="ada2",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [12]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [14]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import HTMLNodeParser, SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import set_global_handler

from llama_index.core.node_parser import SemanticSplitterNodeParser

from pathlib import Path

reader = SimpleDirectoryReader("test_data/")

docs = reader.load_data()


pipeline = IngestionPipeline(
    transformations=[
        HTMLNodeParser.from_defaults(),
        SemanticSplitterNodeParser(
            buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
        ),
        embed_model,
    ],
)
nodes = pipeline.run(documents=docs)

INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 401 Unauthorized"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 401 Unauthorized"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 401 Unauthorized"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 401 Unauthorized"
Retrying llama_index.embeddings.openai.base.get_embeddings in 0.5563280244747709 seconds as it raised AuthenticationError: Error code: 401 - {'statusCode': 401, 'message': 'Unauthorized. Access token is missing, invalid, audience is incorrect (https://cognitiveservices.azure.com), or have expired.'}.
Retrying llama_index.embeddings.openai.

KeyboardInterrupt: 

In [7]:
print(len(nodes))

2


In [8]:
for node in nodes: 
    print(node)
    print("-----------------------")

Node ID: 65216a67-fd34-42f8-9cad-5af28e60a028
Text: 結婚12年目「倖田來未」が明かす“夫と息子”との私生活　「親子でプロレスにハマってます」
-----------------------
Node ID: 1a7470e9-d50d-46ee-a592-9d0b4394d6f0
Text: 今年でデビュー23年目を迎える歌手の倖田來未（40）。「前編」では、下積み時代から芸能界における浮き沈みを経験しながら、「エロか
っこいい」と称される人気歌手に上り詰めるまでの半生を聞いた。そんな彼女は1月18日にMusic ＆     Live Package「WIN
GS」をリリースした。収録曲の作曲には夫も参加しているが、今回の「後編」では主に夫や息子など家族がアーティスト・倖田來未にどのような影響を与
えているかをインタビューした。 倖田來未が1月18日に発売したMusic＆Live     Package「WINGS」のメイントラックとな
るミディアムバラード「Wings」のクレジットを見ると、作曲者はHi－yunk（ハイユンク）とある。これは、倖田の夫であるKENJI03の作
家名義...
-----------------------


In [9]:
# Indexing 
index = VectorStoreIndex(nodes)

In [12]:
# Store database
from llama_index.core import StorageContext, load_index_from_storage

index.storage_context.persist(persist_dir="database/")

In [13]:
# load database 
# from llama_index.core import StorageContext, load_index_from_storage

# # rebuild storage context
# storage_context = StorageContext.from_defaults(persist_dir="<persist_dir>")

# # load index
# index = load_index_from_storage(storage_context)


In [16]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor


query_engine = index.as_query_engine()
response = query_engine.query(
    "倖田來未さんは結婚して何年目ですか？"
)
print(response)

INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
12年目


In [21]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query("倖田來未さんは結婚して何年目ですか？")
print(response)

INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/emb3large/embeddings?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://artirst-assistant.openai.azure.com//openai/deployments/35instruct/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"
12年目
